# 🔍🦋 Vyhledávání obrázků pomocí CLIP (text → obrázek) s uloženými embeddingy
Tento notebook:
- načítá embeddingy obrázků ze souboru `.pt`
- umožňuje zadat textový dotaz přes interaktivní políčko
- zobrazuje 5 nejpodobnějších obrázků
- má správné pořadí: nejprve widgety, pak funkce, pak propojení tlačítka

In [ ]:
# 📁 Připojení Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 📦 Instalace knihoven
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# 🧠 Načtení CLIP modelu a embeddingů
import clip
import torch
from PIL import Image
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, preprocess = clip.load("ViT-B/32", device=device)

# Načtení uložených embeddingů
embedding_path = "/content/drive/MyDrive/custom_vision/clip_image_embeddings.pt"
data = torch.load(embedding_path, map_location=device)
image_embeddings = data['embeddings']
images = [Path(p) for p in data['paths']]
print(f"✅ Načteno {len(images)} obrázků a embeddingů.")

In [ ]:
# ✏️ Interaktivní widgety
import ipywidgets as widgets
from IPython.display import display, clear_output

text_widget = widgets.Text(
    value='motýl s průhlednými křídly',
    placeholder='Zadej textový dotaz...',
    description='Dotaz:',
    layout=widgets.Layout(width='70%')
)

search_button = widgets.Button(
    description='Vyhledat',
    button_style='info',
    tooltip='Klikni pro vyhledání',
    icon='search'
)

display(text_widget, search_button)

In [ ]:
# 🔍 Funkce pro vyhledání podobných obrázků
def show_results(_):
    clear_output(wait=True)
    display(text_widget, search_button)

    text_query = text_widget.value
    print(f"🔍 Výsledky pro dotaz: '{text_query}'\n")

    text = clip.tokenize([text_query]).to(device)
    with torch.no_grad():
        text_embedding = model.encode_text(text)

    similarities = (image_embeddings @ text_embedding.T).squeeze(1)
    values, indices = similarities.topk(5)

    for rank, i in enumerate(indices.cpu().tolist()):
        img_path = images[i]
        similarity = values[rank].item()
        print(f"{rank+1}. 🖼️ {img_path.name} (similarita: {similarity:.4f})")
        display(Image.open(img_path).resize((300, 300)))

In [ ]:
# 🔗 Propojení tlačítka s funkcí
search_button.on_click(show_results)